<a href="https://colab.research.google.com/github/dkbs12/External_test/blob/main/extractiveQA_original_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,elasticsearch,inference,ocr,preprocessing,file-conversion,pdf]

apt install libgraphviz-dev
pip install pygraphviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 106.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 57.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.0 MB/s eta 0:00:00
  Prepa

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.8.0 which is incompatible.




In [2]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [3]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [4]:
# Connect to Elasticsearch
import os
import time

from haystack.document_stores.elasticsearch import ElasticsearchDocumentStore

# Wait 30 seconds only to be sure Elasticsearch is ready before continuing
time.sleep(30)

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")

document_store = ElasticsearchDocumentStore(host=host, username="", password="", index="document")

In [5]:
from haystack.utils import fetch_archive_from_http, convert_files_to_docs
from haystack.nodes import PreProcessor

doc_dir = "data/Phase1_test_data_02"
url = "https://github.com/dkbs12/External_test/raw/main/Phase1_test_data.zip"
fetch_archive_from_http(url=url, output_dir=doc_dir)

# convert files to dicts containing documents that can be indexed to our datastore
got_docs = convert_files_to_docs(dir_path=doc_dir)

In [6]:
preprocessor = PreProcessor(
    clean_whitespace=True,
    clean_header_footer=True,
    clean_empty_lines=True,
    split_by="word",
    split_length=200,
    split_overlap=20,
    split_respect_sentence_boundary=True,
)

all_docs = preprocessor.process(got_docs)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Preprocessing: 100%|██████████| 3/3 [00:00<00:00, 31.44docs/s]


In [7]:
document_store.delete_documents()
document_store.write_documents(all_docs)

In [8]:
from haystack.nodes import BM25Retriever, EmbeddingRetriever, FARMReader
from haystack.utils import print_answers

# Initialize Sparse Retriever
bm25_retriever = BM25Retriever(document_store=document_store)

# Initialize embedding Retriever
embedding_retriever = EmbeddingRetriever(
    document_store=document_store, embedding_model="sentence-transformers/multi-qa-mpnet-base-dot-v1"
)
document_store.update_embeddings(embedding_retriever, update_existing_embeddings=False)

# Initialize Reader
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2")

(…)e/main/config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

(…)70bdf8fca0ca826b6b5d16ebc/.gitattributes:   0%|          | 0.00/737 [00:00<?, ?B/s]

(…)ca0ca826b6b5d16ebc/1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(…)abd4f70bdf8fca0ca826b6b5d16ebc/README.md:   0%|          | 0.00/8.65k [00:00<?, ?B/s]

(…)d4f70bdf8fca0ca826b6b5d16ebc/config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

(…)quad2/resolve/main/tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

(…)erta-base-squad2/resolve/main/vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

(…)erta-base-squad2/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)ad2/resolve/main/special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [9]:
from haystack.pipelines import Pipeline
from haystack.nodes import JoinDocuments

# Create ensembled pipeline
p_ensemble = Pipeline()
p_ensemble.add_node(component=bm25_retriever, name="BM25Retriever", inputs=["Query"])
p_ensemble.add_node(component=embedding_retriever, name="EmbeddingRetriever", inputs=["Query"])
p_ensemble.add_node(
    component=JoinDocuments(join_mode="concatenate"), name="JoinResults", inputs=["BM25Retriever", "EmbeddingRetriever"]
)
p_ensemble.add_node(component=reader, name="Reader", inputs=["JoinResults"])

# Uncomment the following to generate the pipeline image
# p_ensemble.draw("pipeline_ensemble.png")

# Run pipeline
res = p_ensemble.run(
    query="What is NDC?", params={"EmbeddingRetriever": {"top_k": 10}, "BM25Retriever": {"top_k": 10}, "Reader":{"top_k": 5}}
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.94 Batches/s]


In [10]:
print_answers(res, details="medium")

'Query: What is NDC?'
'Answers:'
[   {   'answer': 'New\nDistribution Capability',
        'context': ' distribution\n'
                   'landscape, shaped by such initiatives as IATA’s New\n'
                   'Distribution Capability (NDC) and a global digital trans-\n'
                   'formation, will dramatical',
        'score': 0.968765914440155},
    {   'answer': 'an XML-based communication standard',
        'context': 'is NDC?\n'
                   'New Distribution Capability or NDC is essentially an '
                   'XML-based communication standard created by the '
                   'International Air Transportation Associat',
        'score': 0.6776788830757141},
    {   'answer': 'allows carriers to take back control of their products '
                  'adjusting offers to current market demand and customer '
                  'preferences',
        'context': 'e NDC approach allows carriers to take back control of '
                   'their products adjust

In [12]:
res = p_ensemble.run(
    query="Who is Dr. Belobaba?", params={"EmbeddingRetriever": {"top_k": 5}, "BM25Retriever": {"top_k": 5}, "Reader":{"top_k": 5}}
)
print_answers(res, details="medium")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  5.22 Batches/s]


'Query: Who is Dr. Belobaba?'
'Answers:'
[   {   'answer': 'if demand is reaching capacity or is greater than capacity',
        'context': 'o Dr. Belobaba, continuous pricing works best if demand is '
                   'reaching capacity or is greater than capacity. As airlines '
                   'prepare for the post-COVID recov',
        'score': 0.8851358890533447},
    {   'answer': 'Principal Research Scientist',
        'context': 'namic offer generation and\n'
                   'continuous pricing, Dr. Belobaba, Principal Research '
                   'Scientist in the International Center for Air\n'
                   'Transportation at MIT an',
        'score': 0.6438376307487488},
    {   'answer': 'Principal Research Scientist',
        'context': 'n Massachusetts Institute of Technology\n'
                   'Peter P. Belobaba is Principal Research Scientist in the '
                   'International Center for Air Transportation at the Ma',
        'score': 0.55602294